In [2]:
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from tensorflow.python.keras.layers import *
from tensorflow.python.keras.models import Model
import numpy as np 
import pandas as pd 
import re
import nltk
from preprocess import *
# from models import *

2022-12-01 05:00:07.236228: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-01 05:00:08.682702: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:
2022-12-01 05:00:08.682788: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:
2022-12-01 05:00:08.682795: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] 

In [3]:
df = pd.read_csv("questions.csv")
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [4]:
# question_1, question_2 = df['question1'].to_list(), df['question2'].to_list()
# is_duplicate = df['is_duplicate'].to_list()
# preprocess_neural(question_1, question_2, is_duplicate)

In [5]:
df1 = pd.read_csv("preprocessed_neural.csv")
q1_preprocessed, q2_preprocessed, is_duplicate = df1['question1'].to_list(), df1['question2'].to_list(), df1['is_duplicate'].to_list()

In [6]:
df1.head()

,question1,question2,is_duplicate
0,step step guide invest share market india,step step guide invest share market,0
1,story kohinoor kohinoor diamond,would happen indian government stole kohinoor ...,0
2,increase speed internet connection using vpn,internet speed increased hacking dns,0
3,mentally lonely solve,find remainder math2324math divided 2423,0
4,one dissolve water quikly sugar salt methane c...,fish would survive salt water,0


Acquired Test data

In [7]:
MAX_NB_WORDS = 200000
tokenizer = Tokenizer(num_words = MAX_NB_WORDS)
tokenizer.fit_on_texts(list(df1['question1'].values.astype(str))+list(df1['question2'].values.astype(str)))

In [8]:

q1_sequence = tokenizer.texts_to_sequences(df['question1'].values.astype(str))
q1_sequence = pad_sequences(q1_sequence, maxlen = 30, padding='post')

q2_sequence = tokenizer.texts_to_sequences(df['question2'].values.astype(str))
q2_sequence = pad_sequences(q2_sequence, maxlen = 30, padding='post')

In [9]:
windex = tokenizer.word_index

In [10]:
embedding_index = {}
with open('glove.6B.300d.txt','r') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vectors = np.asarray(values[1:], 'float32')
        embedding_index[word] = vectors
    f.close()

In [11]:
embedding_matrix = np.random.random((len(windex)+1, 300))

for word, i in windex.items():
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

print(embedding_matrix.shape)

(108101, 300)


In [12]:
q1_embeddings = []
for i in range(len(q1_sequence)):
    embedding = np.zeros(300)
    for j in range(len(q1_sequence[i])):
        embedding += embedding_matrix[q1_sequence[i][j]]
    embedding /= len(q1_sequence[i])
    q1_embeddings.append(embedding)

q2_embeddings = []
for i in range(len(q2_sequence)):
    embedding = np.zeros(300)
    for j in range(len(q2_sequence[i])):
        embedding += embedding_matrix[q2_sequence[i][j]]
    embedding /= len(q2_sequence[i])
    q2_embeddings.append(embedding)

In [13]:
#split the data into 70-20-10 train-validation-test with random state 42
from sklearn.model_selection import train_test_split
q1_train, q1_test, q2_train, q2_test, y_train, y_test = train_test_split(q1_embeddings, q2_embeddings, is_duplicate, test_size=0.1, random_state=42)
q1_train, q1_val, q2_train, q2_val, y_train, y_val = train_test_split(q1_train, q2_train, y_train, test_size=0.222, random_state=42)

In [14]:
#print the ratio of positive and negative samples in train, validation and test
print("Train: ", sum(y_train)/len(y_train))
print("Validation: ", sum(y_val)/len(y_val))
print("Test: ", sum(y_test)/len(y_test))

Train:  0.36970242825607064
Validation:  0.3677435326154227
Test:  0.369077060045504


In [26]:
def concatenate_embeddings(q1_embeddings, q2_embeddings):
    embeddings = np.zeros((len(q1_embeddings), 600))
    for i in range(len(q1_embeddings)):
        embeddings[i] = np.concatenate((q1_embeddings[i], q2_embeddings[i]))
xtrain_concat = concatenate_embeddings(q1_train, q2_train)
xval_concat = concatenate_embeddings(q1_val, q2_val)
xtest_concat = concatenate_embeddings(q1_test, q2_test)

In [27]:
print(xval_concat.shape)
print(xtrain_concat.shape)

AttributeError: 'NoneType' object has no attribute 'shape'

In [28]:
# %load models.py
from tensorflow.python.keras.layers import *
from tensorflow.python.keras.models import Model
import tensorflow as tf
import numpy as np 
from keras import Sequential

class NeuralModels:
    def __init__(self, emb_mat, vocab_size = -1, loss = "binary_crossentropy", epochs = 10, optimizer = "adam", metrics = ["accuracy"]):
        self.epochs = epochs
        self.optimizer = optimizer
        self.loss = loss
        self.metrics = metrics
        self.vocab_size = vocab_size
        self.embedding_matrix = emb_mat
        self.model = Sequential()

class CBOW(NeuralModels):
    def __init__(self, emb_mat, vocab_size = -1, loss = "binary_crossentropy", epochs = 10, optimizer = "adam", metrics = ["accuracy"]):
        super().__init__(emb_mat, vocab_size, loss, epochs, optimizer, metrics)
    
    def fit(self,xtrain, xval, ytrain, yval):
        self.model.add(Dense(300, input_shape = (600,), activation = "relu"))
        self.model.add(Dropout(0.1))
        self.model.add(Dense(200, activation = "relu"))
        self.model.add(Dropout(0.1))
        self.model.add(Dense(100, activation = "relu"))
        self.model.add(Dropout(0.1))
        self.model.add(Dense(1, activation = "softmax"))
        self.model.compile(loss = self.loss, optimizer = self.optimizer, metrics = self.metrics)
        self.model.fit(xtrain, ytrain, epochs = self.epochs, validation_data = (xval, yval), batch_size = 64, verbose = 1)

    def get_model_summary(self):
        self.model.summary()
    
    def predict(self, xtest):
        return self.model.predict(xtest)


In [29]:
model = CBOW(embedding_matrix, embedding_matrix.shape[0], loss="binary_crossentropy")

In [30]:
xtrain_concat, xval_concat, y_train, y_val = np.array(xtrain_concat), np.array(xval_concat), np.array(y_train), np.array(y_val)
model.fit(xtrain_concat, xval_concat, y_train, y_val)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type NoneType).